In [13]:
# Import libraries
import torch
from torch import tensor
from torch import nn 
from torch import sigmoid
from torch import atan
from torch import tanh
import pandas as pd
import torch.optim as optim
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [14]:
class Model(nn.Module): # Design your model using class
  def __init__(self):    
    super(Model, self).__init__() #In the constructor, we instantiate nn.Linear module.
    self.linear1 = nn.Linear(12, 100, bias=True).cuda() # nn.Linear(<input size> ,<output size>)
    self.linear2 = nn.Linear(100, 100, bias=True).cuda()
    self.linear3 = nn.Linear(100, 42, bias=True).cuda()

  def forward(self, x):
    x = atan(self.linear1(x))
    x = atan(self.linear2(x))
    x = atan(self.linear2(x))
    y_pred = self.linear3(x)
    return y_pred
model=Model() # our model

In [15]:
model.load_state_dict(torch.load('saved_model_state.pt'))

<All keys matched successfully>

In [16]:
def generate_noise(a, b):
  low,high = 0,1 #range of uniform distribution

  x = torch.distributions.uniform.Uniform(low,high).sample([a,b]).cuda()
  return x

In [17]:
class Generator(nn.Module):

    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), 48).cuda()
        self.dense_layer1 = nn.Linear(48, 96).cuda()
        self.dense_layer2 = nn.Linear(96, int(input_length)).cuda()
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x1 = self.activation(self.dense_layer(x))
        x2 = self.activation(self.dense_layer1(x1))
        x3 = self.activation(self.dense_layer2(x2))
        return x3

In [18]:
import random
seed = 7777  
random.seed(seed)  
np.random.seed(seed)  
torch.manual_seed(seed)
generator = Generator(7)
generator.requires_grad_= True

In [19]:
def scaler (value, min , max):
  scaled_value = value*(max - min) + min
  return scaled_value

In [20]:
lpy_min = 50 #mm
lpy_max = 2050 #mm
lpx_min = 50 #mm
lpx_max = 650 #mm
wp_min = 25 # mm
wp_max = 325 # mm
a_min = 0 # mm
a_max = 200 # mm
p_min = 0 # mm
p_max = 200 # mm
ls_min = 50 # mm
ls_max = 450 # mm
ws_min = 25 # mm
ws_max = 225 # mm
# Ip_min = 50 # A rms
# Ip_max = 200 # A rms
# Np_min = 4 #turn
# Np_max = 10 #turn
# Ns_min = 4 #turn
# Ns_max = 10 #turn

In [21]:
scaler_array = [50,2050,50,650,25,325,0,200,0,200,50,450,25,225]

In [22]:
def scale_and_extract(arr):
  for i in range(arr.shape[1]):
    arr [:,i] = scaler(arr[:,i], scaler_array[2*i], scaler_array[2*i+1]).reshape(1, arr.shape[0])
  y0_i = (arr[:,0]+arr[:,2])/2
  y1_i = 3/2*arr[:,0] + 5/2*arr[:,2] + 2*arr[:,3]+arr[:,4] 

  ys_min = (y1_i).reshape(1, arr.shape[0])
  ys_max = (y1_i+(y1_i-y0_i)/2).reshape(1,arr.shape[0])
  ys = None
  for i in range(arr.shape[0]):
    if i == 0:
      ys = torch.linspace(ys_min[0][i].item(),ys_max[0][i].item(), steps = 5)

    else:
      temp = torch.linspace(ys_min[0][i].item(),ys_max[0][i].item(), steps = 5)
      ys = torch.vstack((ys, temp))

  ys = ys.cuda()
  ys0_i = ys[:,0].reshape(arr.shape[0],1)
  ys1_i = ys[:,1].reshape(arr.shape[0],1)
  ys2_i = ys[:,2].reshape(arr.shape[0],1)
  ys3_i = ys[:,3].reshape(arr.shape[0],1)
  ys4_i = ys[:,4].reshape(arr.shape[0],1)

  arr = torch.hstack((arr, ys0_i, ys1_i, ys2_i, ys3_i, ys4_i))

  return arr



In [23]:
TRAINING = 100
learning_rate = 0.01
optimizer = torch.optim.SGD(generator.parameters(), lr = learning_rate)

In [24]:
for i in range(TRAINING):

  noise = generate_noise(100,7)
  GP = generator(noise)
  scaled_array = scale_and_extract(GP)
  MP = model(scaled_array)
  kdiff = [abs((x-y)/x)*100 for (x,y) in zip(MP[:,0],MP[:,5])]
  mean = torch.mean(torch.tensor(kdiff, requires_grad=True))
  mean.backward()
  print(mean)
  optimizer.step()
  optimizer.zero_grad()
  #We want to maximize the kdiff


  #KDIFF---
  #k0mm_ys0 is column 15 and k100mm_ys0 is column 20, [0,41] in generated 42 parameters
  #kdiff =[abs((x-y)/x)*100 for (x,y) in zip(k0mm_ys0,k100mm_ys0)] 

  #CALCULATE THE KDIFF AND OPTIMIZE IT
  #BACKPROPAGATE 
  # kdiff.backward()
  # optimizer.step()
  # optimizer.zero_grad()


tensor(20.9466, grad_fn=<MeanBackward0>)
tensor(20.9470, grad_fn=<MeanBackward0>)
tensor(20.9454, grad_fn=<MeanBackward0>)
tensor(20.9481, grad_fn=<MeanBackward0>)
tensor(20.9502, grad_fn=<MeanBackward0>)
tensor(20.9450, grad_fn=<MeanBackward0>)
tensor(20.9523, grad_fn=<MeanBackward0>)
tensor(20.9496, grad_fn=<MeanBackward0>)
tensor(20.9514, grad_fn=<MeanBackward0>)
tensor(20.9479, grad_fn=<MeanBackward0>)
tensor(20.9450, grad_fn=<MeanBackward0>)
tensor(20.9469, grad_fn=<MeanBackward0>)
tensor(20.9478, grad_fn=<MeanBackward0>)
tensor(20.9467, grad_fn=<MeanBackward0>)
tensor(20.9435, grad_fn=<MeanBackward0>)
tensor(20.9508, grad_fn=<MeanBackward0>)
tensor(20.9464, grad_fn=<MeanBackward0>)
tensor(20.9469, grad_fn=<MeanBackward0>)
tensor(20.9495, grad_fn=<MeanBackward0>)
tensor(20.9459, grad_fn=<MeanBackward0>)
tensor(20.9416, grad_fn=<MeanBackward0>)
tensor(20.9479, grad_fn=<MeanBackward0>)
tensor(20.9487, grad_fn=<MeanBackward0>)
tensor(20.9462, grad_fn=<MeanBackward0>)
tensor(20.9453, 